# Capstone Final Project - The Battle of the Neighborhoods 
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will be working alongside the **Toronto Public Health** Division. Toronto Public Health (TPH) reports to the Board of Health and is responsible for the health and well-being of all 2.9 million residents. TPH has focused on protecting and promoting the health of Toronto residents since 1883 by:

* preventing the spread of disease, promoting healthy living and advocating for conditions that improve health for Toronto residents
* using surveillance to monitor the health status of the population in order to respond to on-going and emerging health needs
* developing and implementing public policy and practices that enhance the health of individuals, communities and the entire city

![alt text](http://www.gtaweekly.ca/wp-content/uploads/2019/07/PublicHealth-678x381.jpeg "Toronto Public Health Logo")

To address the unique needs of their community they wanted to research the effect of **Neighborhood and Built Environment** as social determinants of health and include any convenient measures to their 2020-2024 Strategic Plan. In order to do so we will use data science to determine if there are significant differences between neighborhoods, and if so, generate a cluster of neighborhoods that are in need of more help. 

## Data <a name="data"></a>

Based on the definition of our problem, possible factors that could determine how living in a certain neighborhood could affect health are:
* Poverty
* Access to foods that support healthy eating patterns
* Access to parks, fitness or recreation centers
* Crime and Violence
* Environmental Conditions
* Access to Health Care  

The following data sources will be needed to extract/generate the required information:
* List of Toronto's neighborhoods, previously extracted in this course
* Access to the different services, their type and location in every neighborhood will be obtained using **Foursquare API**
* Income data will be obtained from the **Canadian census**
* Crime rates will be obtained from data from the **Toronto Police Service**
* Air Quality Health Index (AQHI) data will be obtained from the **Ministry of Environment, Conservation and Parks**